# AIRSS Workflow
### Workflow that generates AIRSS metal-chalcogen inorganic structures, charge balances them with H, calculates their energetics, and ranks the structures by their energetics

## Import libraries, global variables

In [174]:
import numpy as np
import os
import ase
import subprocess as sp
from ase.io import read, write
import ase.neighborlist as nl
from ase import Atoms
from pymatgen.io.vasp.inputs import Kpoints, Poscar
from pymatgen.io.vasp.sets import MPRelaxSet
from pymatgen.core import Structure

# path = "/pscratch/sd/a/aladera/airss/AgS/"
path = "/Users/adrianaladera/Desktop/MIT/research/MOChAs/airss/AgS_test/" # local
os.chdir(path)
seed = "AgS"
metal, chalc = 'Ag', 'S'
incar_settings_1 = {"NELMIN": 5, "ALGO": 'Normal', "ISPIN": 1, "ISMEAR": 0, "NSW": 4, "ENCUT": 600, "EDIFF" : 1e-4, "EDIFFG" : -0.001, "KPAR": 1, "NCORE": 32, "SIGMA" : 0.01, "NELM" : 150}  # user custom incar settings
incar_settings_2 = {"NELMIN": 5, "ALGO": 'Normal', "ISPIN": 1, "ISMEAR": 0, "NSW": 20, "ENCUT": 700, "EDIFF" : 1e-6, "EDIFFG" : -0.001, "KPAR": 1, "NCORE": 32, "SIGMA" : 0.01}  # user custom incar settings

In [137]:
cunt = 0
for file in os.listdir(path):
    cunt += 1
    if ".vasp" in file:
        os.rename(f"{path}{file}", f"{path}{cunt}.vasp")
        print()
        # print(file[22:-9])

## Helper Functions

In [185]:

def activate_env():
    sp.run("module load conda", shell=True)
    sp.run("conda activate crystal_gen", shell=True)
    print("crystal_gen activated")

def generate_structs(path, n_structures, seed):
    '''Generates structures with AIRSS and converts from .res to .vasp
    - path: path to main directory storing all AIRSS generated structures
    - n_structures: max number of structures to be generated
    - seed: <seed>.cell, source file for generating structures'''
    sp.run("airss.pl -vasp -build -max {} -seed {}".format(n_structures, seed), shell=True)
    cunt = 1
    for file in os.listdir(path):
        if ".res" in file:
            struct = read(file)
            write("{}-{}.vasp".format(seed, cunt), struct)
            cunt += 1
    sp.run("rm *.res", shell=True)
    print("{} VASP files generated".format(cunt))

def attach_hydrogens(path, metal, chalcogen, met_chalc_dist, chalc_H_dist, file):
    mocha = Structure.from_file("{}{}".format(path,file))
    ind_largest_v = np.argmax(mocha.lattice.abc)
    positions = []
    for atom in range(len(mocha)):
        # print(mocha[atom].coords)
        if str(mocha[atom].species) == "{}1".format(chalcogen):
            print("chalc found")
            pos = np.array(mocha[atom].coords) # coords are Cartesian
            neighbors = mocha.get_neighbors(site = mocha[atom], r = met_chalc_dist)
            for n in neighbors:
                flag = 0
                if str(n.species) == "{}1".format(metal):
                    flag = 1
                    if mocha[atom].coords[ind_largest_v] > n.coords[ind_largest_v]: # chalc above metal
                        # print(1)
                        pos[ind_largest_v] += chalc_H_dist #/ np.max(mocha.lattice.abc) # add H above chalc
                    elif mocha[atom].coords[ind_largest_v] < n.coords[ind_largest_v]: # chalc below metal
                        pos[ind_largest_v] -= chalc_H_dist #/ np.max(mocha.lattice.abc) # add H below chalc
                        # print(2)
                if flag:
                    break
            positions.append(pos)
    for p in positions:
        mocha.append('H', p, coords_are_cartesian=True)
    poscar = Poscar(mocha)
    poscar.write_file("{}{}{}H-{}".format(path, metal, chalcogen, file))

def attach_hydrogens_ase(path, metal, chalcogen, chalc_H_dist, file):
    '''
    - get length of all unit cell vectors, see which one is the longest
    - based on longest, check to see where the S/Se lies in relation to metal (i.e. above or below)
    - this will determine where relative to the inorganic structure that the H should be placed
    '''
    mocha = read("{}{}".format(path,file))
    # print(mocha.get_chemical_symbols())
    ind_largest_v = np.argmax(mocha.get_cell().lengths()) # get index of largest unit cell vector
    neighbors = nl.build_neighbor_list(mocha, 3*np.ones((len(mocha),)))
    # print(len(mocha))
    for atom in range(len(mocha)):
        if mocha.get_chemical_symbols()[atom] == chalcogen: # if is a chalcogen
            print("chalc found")
            pos = np.array(mocha.get_positions()[atom])
            # pos[ind_largest_v] += chalc_H_dist
            for n in neighbors.get_neighbors(atom)[0]:
                flag = 0
                if mocha.get_chemical_symbols()[n] == metal:
                    flag = 1
                    if mocha.get_positions()[atom][ind_largest_v] > mocha.get_positions()[n][ind_largest_v]: #if chalc above metal
                        # print("1")
                        pos[ind_largest_v] += chalc_H_dist
                        # flag = 1
                    elif mocha.get_positions()[atom][ind_largest_v] < mocha.get_positions()[n][ind_largest_v]:
                        print("2")
                        pos[ind_largest_v] -= chalc_H_dist
                        # flag = 1
                if flag:
                    break
            H = Atoms('H', positions=[pos])
            mocha.extend(H)
    write("{}{}{}H-{}".format(path, metal, chalcogen, file), mocha)

def make_kpoints(path, struct, kp_density):
    structure = Structure.from_file("{}{}".format(path, struct))
    kp = Kpoints.automatic_density(structure, kp_density)
    kp.write_file("{}KPOINTS".format(path))

def create_job_dirs(path, seed, kp_density): 
    '''
    - relax 4 x NSW = 4
    - then relax NSW = 20, get energies but no calculations (see Aria's notebook)
    - pick a subset based on energies and how many times each structures are repeated (based on space group)
    - relax subset until convergence + run static; final structures
    '''
    for file in os.listdir(path):
        if ".vasp" in file and seed in file:
            print(file)
            sp.run("mkdir {} ".format(file[:-5]), shell=True)
            sp.run("cp {} {}/POSCAR ".format(file, file[:-5]), shell=True) # POSCAR
            # make_kpoints("{}{}/".format(path, file[:-5]), "POSCAR", kp_density) # KPOINTS
            sp.run("pot".format(file[:-5]), shell=True) # POINTS
            structure = Structure.from_file("{}/POSCAR".format(file[:-5])) # cp POSCAR, make KPOINTS, INCAR
            relax = MPRelaxSet(structure, user_incar_settings=incar_settings_1)
            relax.write_input("{}".format(file[:-5]), potcar_spec=True)
            sp.run("rm {}/POTCAR.spec".format(file[:-5]), shell=True)
        elif ".vasp" in file and seed not in file:
            sp.run("rm {}".format(file), shell=True)

In [186]:
create_job_dirs(path, seed, 2.0)


# read all the .vasp files in the directory and save them as structures
# structures = []
# for d in os.listdir(path):
#     if os.path.isdir(path + d) and seed in d:
#         # print(Structure.from_file("{}{}/POSCAR".format(path, d)))
#         structures.append(Structure.from_file("{}{}/POSCAR".format(path, d)))



AgSH-8.vasp
AgSH-4.vasp
AgSH-9.vasp
AgSH-12.vasp
AgSH-11.vasp
AgSH-10.vasp
AgSH-1.vasp


In [97]:
a = Structure.from_file("4.vasp")
print(a[0].coords[0])

0.7459362833304949


## Main code that calls all the functions
### Do not run until doing another test on NERSC!

In [150]:
# generate structures
generate_structs(path, 10, seed)

# for each structure, take generated structures and attach H, then write to new vasp file
for file in os.listdir(path):
    if "AgS" not in file and ".vasp" in file:
        print(file)
        attach_hydrogens(path, 'Ag', 'S', 3.0, 1.3, file)
        sp.run("rm {}".format(file))
create_job_dirs(path, seed, 2.0)


9.vasp
chalc found
1
chalc found
2
chalc found
1
chalc found
2
chalc found
2
chalc found
1
8.vasp
chalc found
1
chalc found
1
chalc found
1
chalc found
1
chalc found
1
chalc found
1
4.vasp
chalc found
2
chalc found
1
chalc found
1
chalc found
2
chalc found
1
chalc found
2
12.vasp
chalc found
1
chalc found
1
chalc found
1
chalc found
2
chalc found
1
chalc found
1
1.vasp
chalc found
2
chalc found
2
chalc found
1
chalc found
2
chalc found
2
chalc found
2
11.vasp
chalc found
1
chalc found
2
chalc found
1
chalc found
1
chalc found
1
chalc found
2
10.vasp
chalc found
2
chalc found
2
chalc found
1
chalc found
1
chalc found
1
chalc found
1
